In [1]:
! pip install seaborn

In [17]:
 # Libs principais
from model import *
import pandas as pd
# Controle de execução e pastas
import os
# Desativar alguns warnings
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.style.use('ggplot')

def VirtualLaneDetector(x,y,virtual_lane_lim,virtual_lane_width):
    virtual_lane = -1
    count = 0

    for list_point in virtual_lane_lim:
        count = count + 1
        func_virtual_lane = PolygonalToFunction(list_point)
        if (y>=func_virtual_lane(x)-0.5*virtual_lane_width) and (y<=func_virtual_lane(x)+0.5*virtual_lane_width):
            virtual_lane = count
            break

    return virtual_lane

In [5]:
if __name__=="__main__":
    # os.chdir("data_ignore")
    root_file = "data/json"
    all_files = os.listdir(root_file)

    for f in all_files:
        try:
            print(f"Processando {f}")
            model = YoloMicroscopicDataProcessing()
            model.ImportFromJSON(os.path.join(root_file,f))
        
            model_smoothed = model.SmoothingSavGolFilter(window_length=15,polyorder=1) 
            model_smoothed.to_csv(f"data/processed_smoothed/{f.replace('json','csv')}",index=False)
            print(f"Fim {f}")
        except Exception as e:
            print(f"Erro {f}")
            print(e)


Processando 10_A_1.json
Fim 10_A_1.json
Processando 10_A_2.json
Fim 10_A_2.json
Processando 10_A_3.json
Fim 10_A_3.json
Processando 10_A_4.json
Fim 10_A_4.json
Processando 10_A_5.json
Fim 10_A_5.json
Processando 10_B_1.json
Fim 10_B_1.json
Processando 10_B_2.json
Fim 10_B_2.json
Processando 10_B_3.json
Fim 10_B_3.json
Processando 10_B_4.json
Fim 10_B_4.json
Processando 10_B_5.json
Fim 10_B_5.json
Processando 32_A_1.json
Fim 32_A_1.json
Processando 32_A_2.json
Fim 32_A_2.json
Processando 32_A_3.json
Fim 32_A_3.json
Processando 32_A_4.json
Fim 32_A_4.json
Processando 32_A_5.json
Fim 32_A_5.json
Processando 32_B_1.json
Fim 32_B_1.json
Processando 32_B_2.json
Fim 32_B_2.json
Processando 32_B_3.json
Fim 32_B_3.json
Processando 32_B_4.json
Fim 32_B_4.json
Processando 32_B_5.json
Fim 32_B_5.json
Processando 79_A_1.json
Fim 79_A_1.json
Processando 79_A_2.json
Fim 79_A_2.json
Processando 79_A_3.json
Fim 79_A_3.json
Processando 79_A_4.json
Fim 79_A_4.json
Processando 79_A_5.json
Fim 79_A_5.json


In [25]:
# Corredor - Fx 5

f = "10_A_1.json"
model = YoloMicroscopicDataProcessing()
model.ImportFromJSON(f"data/json/{f}")

# Remover dados das extreminades do vídeo
model.df = model.df[model.df['x'].between(3,model.video_width-3)]

# Alterar a frequência da amostra para 10fps
min_frame = model.df[model.frame_column].min()
max_frame = model.df[model.frame_column].max()
list_frames = list(range(min_frame,max_frame+1,3))
model.df = model.df[model.df[model.frame_column].isin(list_frames)]

# Dataframe de motocicletas
df_motorcycle = model.df[model.df[model.vehicle_type_column].isin(['Moto'])].sort_values([model.frame_column,model.id_column])
df_motorcycle.insert(0,'id_voo',f)

# Definição das faixas de tráfego
# Centroide (y) dentro da região cujo centroo é o limite entre as faixas
# Largura teórica do corredor (metros), corresponde a largura de 2 motos
virtual_lane_width = 1.6
# Corredor sem faixa azul
virutal_lane_group = {'Corredor Principal':[5]}
virutal_lane_group["Outros Corredores"] = [i for i in  range(1,len(model.virtual_lane_lim)+1) if i not in virutal_lane_group['Corredor Principal']]

# Qual corredor virual pertence (-1 para nenhum corredor)
df_motorcycle['virutal_lane'] = df_motorcycle.apply(lambda row:VirtualLaneDetector(row[model.x_centroid_column],row[model.y_centroid_column],model.virtual_lane_lim,virtual_lane_width),axis=1)
# Corredores não classificados recebem 0
df_motorcycle['zero_temp'] = ((-df_motorcycle['virutal_lane'].isin(JoinList(list(virutal_lane_group.values())))) & (df_motorcycle['virutal_lane']!=-1))
df_motorcycle['virutal_lane'] = df_motorcycle.apply(lambda x:x['virutal_lane'] if not x['zero_temp'] else 0,axis=1)
df_motorcycle = df_motorcycle.drop(columns=['zero_temp'])

# Tipo/nome do corredor
df_motorcycle['virtual_lane_type'] = np.nan
for key,value in virutal_lane_group.items():
    df_motorcycle['virtual_lane_type'] = df_motorcycle.apply(lambda x:key if x['virutal_lane'] in value else x['virtual_lane_type'],axis=1)
# Se for -1, estava na mais centralizado na faixa de tráfefo misto
df_motorcycle['virtual_lane_type'] = df_motorcycle.apply(lambda x:'Fora do Corredor' if x['virutal_lane']==-1 else x['virtual_lane_type'],axis=1)
# Se estava em outras faixas não avaliadas
df_motorcycle['virtual_lane_type'] = df_motorcycle.apply(lambda x:'Outro Corredor' if x['virutal_lane']==0 else x['virtual_lane_type'],axis=1)

# Condição do tráfego
speed_ref = '85%'
agg_traffic_state = model.df[-model.df[model.vehicle_type_column].isin(['Moto'])].groupby(['frame'])['instant_speed'].describe(percentiles=[0.85])
agg_traffic_state = agg_traffic_state.reset_index(drop=False)
df_motorcycle = df_motorcycle.merge(agg_traffic_state[['frame',speed_ref]],on='frame',how='left')
df_motorcycle['traffic_condition_speed'] = df_motorcycle[speed_ref]*3.6
df_motorcycle = df_motorcycle.drop(columns=[speed_ref])
df_motorcycle['traffic_condition'] = df_motorcycle['traffic_condition_speed'].apply(lambda x:'Congestionado' if x<5 else 'Não Congestionado')

# Distância lateral
side_distance = pd.DataFrame()
for motorcycle_id in df_motorcycle[model.id_column].unique().tolist():
    instant_list = df_motorcycle[df_motorcycle[model.id_column]==motorcycle_id].sort_values(model.frame_column)['instant'].tolist()
    df_side = pd.concat([model.SideVehicle(
        motorcycle_id,
        t,
        overlap_lon=0.3,
        overlap_lat=0.3,
        report_just_min=True,
        report_nan=True
        ) for t in instant_list],ignore_index=True)
    df_side = df_side.rename(columns=dict(zip(df_side.columns, [i+'_vehicle_side' if 'speed' in i else i for i in df_side.columns])))
    df_side.insert(0,'id_motorcycle',motorcycle_id)
    side_distance = pd.concat([side_distance,df_side],ignore_index=True)

side_distance = side_distance.rename(columns={'id':'id_vehicle_side'})
side_distance[model.global_id_column] = side_distance['id_motorcycle'].astype(str) + '@' + side_distance[model.frame_column].astype(str)
df_motorcycle = df_motorcycle.merge(side_distance[[
    'global_id',
    'id_vehicle_side',
    'x_instant_speed_vehicle_side',
    'y_instant_speed_vehicle_side',
    'instant_speed_vehicle_side',
    'lateral_distance_between_vehicles',
    'side'
    ]],on=model.global_id_column,how='left')

df_motorcycle['delta_speed_vehicle_side'] = df_motorcycle[model.instant_speed_column] - df_motorcycle[model.instant_speed_column+'_vehicle_side']
df_motorcycle['delta_speed_vehicle_side'] = df_motorcycle['delta_speed_vehicle_side']

TypeError: YoloMicroscopicDataProcessing.SideVehicle() got an unexpected keyword argument 'report_just_min'

In [ ]:
df_motorcycle

,id_voo,frame,p1xbb,p1ybb,p2xbb,p2ybb,id,conf_YOLO,vehicle_type,traffic_lane,...,head,global_id,x_instant_speed,y_instant_speed,instant_speed,x_instant_acc,y_instant_acc,instant_acc,virutal_lane,virtual_lane_type
3,10_A_1.json,3,48.889122,31.469154,51.271873,32.376097,3,0.855737,Moto,8.0,...,51.271873,3@3,-20.545561,0.264838,20.547268,-0.339370,-0.047733,0.342711,-1,Fora do Corredor
0,10_A_1.json,3,17.332642,7.721797,19.839833,8.575720,5,0.849500,Moto,3.0,...,19.839833,5@3,16.198400,0.117579,16.198827,0.316796,-0.165767,0.357545,-1,Fora do Corredor
15,10_A_1.json,6,46.836472,31.497223,49.203184,32.399354,3,0.889086,Moto,8.0,...,49.203184,3@6,-20.585013,0.256458,20.586610,-0.380621,-0.069662,0.386943,-1,Fora do Corredor
12,10_A_1.json,6,18.957038,7.735832,21.462224,8.584141,5,0.873780,Moto,3.0,...,21.462224,5@6,16.226570,0.107784,16.226928,0.280952,-0.097000,0.297226,-1,Fora do Corredor
27,10_A_1.json,9,44.783822,31.525292,47.134494,32.422611,3,0.865610,Moto,8.0,...,47.134494,3@9,-20.624465,0.248078,20.625957,-0.421872,-0.091591,0.431700,-1,Fora do Corredor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227162,10_A_1.json,24609,14.286526,27.295598,16.664287,28.106481,2473,0.000000,Moto,NaN,...,16.664287,2473@821.12,-26.232558,-0.082552,26.232688,-0.223895,0.091657,0.241929,7,Outros Corredores
227178,10_A_1.json,24612,11.696461,27.288112,14.110059,28.102852,2473,0.419230,Moto,7.0,...,14.110059,2473@24612,-26.132121,-0.073766,26.132225,-0.108857,0.164308,0.197097,7,Outros Corredores
227196,10_A_1.json,24615,9.079858,27.273142,11.543130,28.101265,2473,0.796804,Moto,7.0,...,11.543130,2473@24615,-26.061648,-0.048281,26.061693,0.274300,0.237785,0.363018,7,Outros Corredores
227211,10_A_1.json,24618,6.452513,27.267261,8.957990,28.111374,2473,0.458318,Moto,7.0,...,8.957990,2473@24618,-26.040790,-0.013248,26.040793,0.694718,0.278969,0.748637,7,Outros Corredores


In [24]:
df_motorcycle.groupby("virtual_lane_type")["id_voo"].value_counts()

virtual_lane_type   id_voo     
Corredor Principal  10_A_1.json    1241
Fora do Corredor    10_A_1.json    3769
Outros Corredores   10_A_1.json    1746
Name: count, dtype: int64